In [ ]:
pip install tldextract

     |████████████████████████████████| 87 kB 3.1 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
import math
import sqlite3
import tldextract

import pandas as pd
import numpy as np
from pandas import *

import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter

MODE_Normal = "(Normal mode)"
MODE_DNT = "(DNT mode)"
TITLE_REQUESTS = "websites with third-party HTTP(S) requests"
TITLE_COOKIES = "websites with third-party cookies"
TITLE_JAVASCRIPT = "websites with third-party JavaScript API calls"

DB_DNT = "drive/MyDrive/CSC533/Data/crawl-data-DNT.sqlite"
DB_Normal = "drive/MyDrive/CSC533/Data/crawl-data.sqlite"
PATH_CSV = "drive/MyDrive/CSC533/Data/urls.csv"

QUERY_REQUESTS = "SELECT visit_id, url FROM http_requests"
QUERY_COOKIES = "SELECT visit_id, host FROM javascript_cookies"
QUERY_JAVASCRIPT = "SELECT visit_id, script_url FROM javascript"




def get_sites_from_csv():
    data = read_csv(PATH_CSV)
    sites = data['URL'].tolist()
    return sites


def query_from_db(query, db_path):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute(query)

    rows = c.fetchall()

    c.close()
    conn.close()

    return rows


def is_third_party(domain, url):
    domain_ext = tldextract.extract(domain)
    url_ext = tldextract.extract(url)
    # print("domain", domain_ext)
    # print("url",url_ext)
    if '.'.join(domain_ext[:]) == '.'.join(url_ext[:]):
        return False

    return True





def count_third_parties(websites, third_parties, rows):
    for row in rows:
        visit_id = row[0]
        url = row[1]
        ext = tldextract.extract(url)
        url = '.'.join(ext[1:])

        if is_third_party(visit_id_site_urls_Normal[visit_id][0], url):
            third_parties.append(url)
            websites.append(visit_id_site_urls_Normal[visit_id])

def count_third_parties_DNT(websites, third_parties, rows):
    for row in rows:
        visit_id = row[0]
        url = row[1]
        ext = tldextract.extract(url)
        url = '.'.join(ext[1:])

        if is_third_party(visit_id_site_urls_DNT[visit_id][0], url):
            third_parties.append(url)
            websites.append(visit_id_site_urls_DNT[visit_id])

def get_visit_id_site_url_mapping():
    visit_id_site_urls_Normal = {}
    visit_id_site_urls_DNT = {}
    for visit_id, site_url in sqlite3.connect(DB_Normal).execute(
            "SELECT visit_id, site_url FROM site_visits"):
        visit_id_site_urls_Normal[visit_id] = site_url
    print ("Normal", len(visit_id_site_urls_Normal), "mappings")
    print ("Distinct site urls", len(set(visit_id_site_urls_Normal.values())))
    print(visit_id_site_urls_Normal)
    
    for visit_id, site_url in sqlite3.connect(DB_DNT).execute(
            "SELECT visit_id, site_url FROM site_visits"):
        visit_id_site_urls_DNT[visit_id] = site_url
    print ("DNT", len(visit_id_site_urls_DNT), "mappings")
    print ("Distinct site urls", len(set(visit_id_site_urls_DNT.values())))
    print(visit_id_site_urls_DNT)
    return visit_id_site_urls_Normal, visit_id_site_urls_DNT





def get_rows(measurement):
    if measurement == "requests":
        return query_from_db(QUERY_REQUESTS, DB_DNT), query_from_db(QUERY_REQUESTS, DB_Normal)
    elif measurement == "cookies":
        return query_from_db(QUERY_COOKIES, DB_DNT), query_from_db(QUERY_COOKIES, DB_Normal)
    elif measurement == "javascript":
        return query_from_db(QUERY_JAVASCRIPT, DB_DNT), query_from_db(QUERY_JAVASCRIPT, DB_Normal)
    else:
        raise Exception('Invalid argument - ' + measurement)


def get_title(measurement):
    if measurement == "requests":
        return TITLE_REQUESTS
    elif measurement == "cookies":
        return TITLE_COOKIES
    elif measurement == "javascript":
        return TITLE_JAVASCRIPT


In [ ]:
visit_id_site_urls_Normal, visit_id_site_urls_DNT = get_visit_id_site_url_mapping()

Normal 107 mappings
Distinct site urls 106
{111825142126748: 'https://games.wanmei.com/', 200948218764997: 'https://news.cctv.com/', 265073816334436: 'http://www.people.com.cn/', 462414623615075: 'https://www.private.com/', 477694907051847: 'https://xvideos.com', 731390832135748: 'https://www.naughtyamerica.com/', 1025904800409066: 'https://pinterest.com', 1107793373515580: 'https://bbc.com', 1164631717281309: 'https://sciencedaily.com', 1188153339145622: 'https://nintendo.com', 1438249824976050: 'https://www.jd.com/?country=USA', 1459659564781047: 'https://nordstrom.com', 1561989566078617: 'https://www.newsensations.com/', 1610374238626312: 'https://game.qq.com/', 1672346515268389: 'https://www.brazzersnetwork.com/home', 1715945945859022: 'https://fuq.com', 1717661855529999: 'https://instagram.com', 1733844507669164: 'https://tumblr.com', 1779511345460382: 'https://twitter.com', 2110162006085513: 'https://usatoday.com', 2133861977171269: 'https://news.qq.com/', 2164175389863695: 'http

In [ ]:
def count_for_each(rows, websites,data):
  count = 0
  if data == "DNT":
    for row in rows:
          visit_id = row[0]
          url = row[1]
          websites[visit_id_site_urls_DNT[visit_id]] += 1
          count += 1
    print("DNT total", count)
    
  else:
    for row in rows:
      visit_id = row[0]
      url = row[1]
      websites[visit_id_site_urls_Normal[visit_id]] += 1
      count += 1
    print("Normal total", count)


def get_counts(measurement):
  DNT_rows, Normal_rows = get_rows(measurement)
  DNT_websites = {}
  Normal_websites = {}
  DNT_websites = dict.fromkeys(sites, 0)
  Normal_websites = dict.fromkeys(sites, 0)


  count_for_each(DNT_rows, DNT_websites, "DNT")
  count_for_each(Normal_rows, Normal_websites, "Normal")
  return DNT_websites, Normal_websites

def combine_results(DNT, Normal):
  result_list = dict.fromkeys(sites, [])

  for r in result_list:
    result_list[r] = [Normal[r], DNT[r]]
  return result_list

def print_result(d):
  print ("{:<40} {:<15} {:<10} ".format("Website", "Before DNT", "After DNT"))
  for k, v in d.items():
    normal, DNT = v
    print ("{:<40} {:<15} {:<10} ".format(k, normal, DNT))

def find_difference(d):
  result_list = {}
  for k, v in d.items():
    normal, DNT = v
    if (normal - DNT) > 0.5*normal:
      result_list[k] = normal - DNT
  return result_list

def find_top(d):
  result_list = {}
  for k, v in d.items():
    normal, DNT = v
    if normal != 0:
      result_list[k] = (normal - DNT)/normal
    else:
      result_list[k] = 0
  top = sorted(result_list, key=result_list.get, reverse=True)[:10]
  top_list = {}
  for t in top:
    top_list[t] = result_list[t]
  return top_list

sites = get_sites_from_csv()
DNT_requests, Normal_requests = get_counts("requests")
DNT_cookies, Normal_cookies = get_counts("cookies")
DNT_js, Normal_js = get_counts("javascript")

requests = combine_results(DNT_requests, Normal_requests)
print("Total Request")
# print_result(requests)
# print(len(find_difference(requests)))
print(find_top(requests))


cookies = combine_results(DNT_cookies, Normal_cookies)
print("Total Cookie")
# print_result(cookies)
# print(len(find_difference(cookies)))
# print(find_difference(cookies))
print(find_top(cookies))

js = combine_results(DNT_js, Normal_js)
print("Total JavaScript")
# print_result(js)
# print(len(find_difference(js)))
# print(find_difference(js))
print(find_top(js))




DNT total 17209
Normal total 17540
DNT total 7955
Normal total 8996
DNT total 63882
Normal total 69302
Total Request
{'https://sciencedaily.com': 0.54, 'https://pornhub.com': 0.375, 'https://5237.xg4ken.com/trk/v1?prof=529&camp=52825&affcode=cr16578&kct=google&kchid=3518500093&cid=544484090159%7C3335749%7Cyahoo%20politics&mType=b&networkType=search&kdv=c&criteriaid=kwd-297725782280&adgroupid=126103829425&campaignid=14488307422&locphy=9009731&adpos=&url=https://ad.doubleclick.net/ddm/trackclk/N360801.1913355YAHOOADMANAGER/B23818180.312205931;dc_trk_aid=504764009;dc_trk_cid=129322662;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=https://www.yahoo.com/&gclid=Cj0KCQiAhMOMBhDhARIsAPVml-HtEw-gsofJd4xtuA--pddOcMFXfiP1G3wfMh7SoN7Y4fwObFEVA58aArqwEALw_wcB': 0.29838709677419356, 'https://redtube.com': 0.2698412698412698, 'https://www.espn.com': 0.21403508771929824, 'https://blizzard.com': 0.2125, 'https://www.netflix.com': 0.15555555555555556, 'https://nfl.com': 0.1026252983293556, 'https://

In [ ]:
def count_for_each(rows, websites,data):
  count = 0
  if data == "DNT":
    for row in rows:
          visit_id = row[0]
          url = row[1]
          if not is_third_party(visit_id_site_urls_DNT[visit_id], url):
            websites[visit_id_site_urls_DNT[visit_id]] += 1
            count += 1
    print("DNT total", count)
    
  else:
    for row in rows:
      visit_id = row[0]
      url = row[1]
      if not is_third_party(visit_id_site_urls_Normal[visit_id], url):
        websites[visit_id_site_urls_Normal[visit_id]] += 1
        count += 1
    print("Normal total", count)


def get_counts(measurement):
  DNT_rows, Normal_rows = get_rows(measurement)
  DNT_websites = {}
  Normal_websites = {}
  DNT_websites = dict.fromkeys(sites, 0)
  Normal_websites = dict.fromkeys(sites, 0)


  count_for_each(DNT_rows, DNT_websites, "DNT")
  count_for_each(Normal_rows, Normal_websites, "Normal")
  return DNT_websites, Normal_websites

def combine_results(DNT, Normal):
  result_list = dict.fromkeys(sites, [])

  for r in result_list:
    result_list[r] = [Normal[r], DNT[r]]
  return result_list

def print_result(d):
  print ("{:<40} {:<15} {:<10} ".format("Website", "Before DNT", "After DNT"))
  for k, v in d.items():
    normal, DNT = v
    print ("{:<40} {:<15} {:<10} ".format(k, normal, DNT))

sites = get_sites_from_csv()
DNT_requests, Normal_requests = get_counts("requests")
DNT_cookies, Normal_cookies = get_counts("cookies")
DNT_js, Normal_js = get_counts("javascript")

requests = combine_results(DNT_requests, Normal_requests)
print("First-party Request")
print_result(requests)


cookies = combine_results(DNT_cookies, Normal_cookies)
print("First-party Cookie")
print_result(cookies)

js = combine_results(DNT_js, Normal_js)
print("First-party JavaScript")
print_result(js)




DNT total 859
Normal total 855
DNT total 1969
Normal total 1974
DNT total 1539
Normal total 1610
First-party Request
Website                                  Before DNT      After DNT  
https://nytimes.com                      1               1          
https://bbc.com                          1               1          
https://foxnews.com                      1               1          
https://5237.xg4ken.com/trk/v1?prof=529&camp=52825&affcode=cr16578&kct=google&kchid=3518500093&cid=544484090159%7C3335749%7Cyahoo%20politics&mType=b&networkType=search&kdv=c&criteriaid=kwd-297725782280&adgroupid=126103829425&campaignid=14488307422&locphy=9009731&adpos=&url=https://ad.doubleclick.net/ddm/trackclk/N360801.1913355YAHOOADMANAGER/B23818180.312205931;dc_trk_aid=504764009;dc_trk_cid=129322662;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=https://www.yahoo.com/&gclid=Cj0KCQiAhMOMBhDhARIsAPVml-HtEw-gsofJd4xtuA--pddOcMFXfiP1G3wfMh7SoN7Y4fwObFEVA58aArqwEALw_wcB 1               1          
h

In [ ]:
def count_for_each(rows, websites,data):
  count = 0
  if data == "DNT":
    for row in rows:
          visit_id = row[0]
          url = row[1]
          if is_third_party(visit_id_site_urls_DNT[visit_id], url):
            websites[visit_id_site_urls_DNT[visit_id]] += 1
            count += 1
    print("DNT total", count)
    
  else:
    for row in rows:
      visit_id = row[0]
      url = row[1]
      if is_third_party(visit_id_site_urls_Normal[visit_id], url):
        websites[visit_id_site_urls_Normal[visit_id]] += 1
        count += 1
    print("Normal total", count)


def get_counts(measurement):
  DNT_rows, Normal_rows = get_rows(measurement)
  DNT_websites = {}
  Normal_websites = {}
  DNT_websites = dict.fromkeys(sites, 0)
  Normal_websites = dict.fromkeys(sites, 0)


  count_for_each(DNT_rows, DNT_websites, "DNT")
  count_for_each(Normal_rows, Normal_websites, "Normal")
  return DNT_websites, Normal_websites

def combine_results(DNT, Normal):
  result_list = dict.fromkeys(sites, [])

  for r in result_list:
    result_list[r] = [Normal[r], DNT[r]]
  return result_list

def print_result(d):
  print ("{:<40} {:<15} {:<10} ".format("Website", "Before DNT", "After DNT"))
  for k, v in d.items():
    normal, DNT = v
    print ("{:<40} {:<15} {:<10} ".format(k, normal, DNT))

sites = get_sites_from_csv()
DNT_requests, Normal_requests = get_counts("requests")
DNT_cookies, Normal_cookies = get_counts("cookies")
DNT_js, Normal_js = get_counts("javascript")

requests = combine_results(DNT_requests, Normal_requests)
print("Third-party Request")
print_result(requests)


cookies = combine_results(DNT_cookies, Normal_cookies)
print("Third-party Cookie")
print_result(cookies)

js = combine_results(DNT_js, Normal_js)
print("Third-party JavaScript")
print_result(js)




DNT total 16350
Normal total 16685
DNT total 5986
Normal total 7022
DNT total 62343
Normal total 67692
Third-party Request
Website                                  Before DNT      After DNT  
https://nytimes.com                      359             453        
https://bbc.com                          146             143        
https://foxnews.com                      907             837        
https://5237.xg4ken.com/trk/v1?prof=529&camp=52825&affcode=cr16578&kct=google&kchid=3518500093&cid=544484090159%7C3335749%7Cyahoo%20politics&mType=b&networkType=search&kdv=c&criteriaid=kwd-297725782280&adgroupid=126103829425&campaignid=14488307422&locphy=9009731&adpos=&url=https://ad.doubleclick.net/ddm/trackclk/N360801.1913355YAHOOADMANAGER/B23818180.312205931;dc_trk_aid=504764009;dc_trk_cid=129322662;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=https://www.yahoo.com/&gclid=Cj0KCQiAhMOMBhDhARIsAPVml-HtEw-gsofJd4xtuA--pddOcMFXfiP1G3wfMh7SoN7Y4fwObFEVA58aArqwEALw_wcB 123             86     